In [29]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras.models import Model
import os
import keras

# path to the model weights files.
weights_path = '../keras/examples/vgg16_weights.h5'
#weights_path = '../model/vgg16_weights.h5'
top_model_weights_path = '../model/bottleneck_fc_model.h5'
#top_model_weights_path = '../model/fc_model.h5'
img_width, img_height = 150, 150

#full
#train_data_dir = '../data/Pandora18K_train_val_test_split/train'
#validation_data_dir = '../data/Pandora18K_train_val_test_split/val'
#nb_train_samples = 14313
#nb_validation_samples = 1772
#total_num_classes = 18

#small
train_data_dir = '../data/Pandora18K_small_train_val_test_split/train'
validation_data_dir = '../data/Pandora18K_small_train_val_test_split/val'
nb_train_samples = 1462
nb_validation_samples = 167
total_num_classes = 18

#verysmall
#train_data_dir = '../data/verysmall/train'
#validation_data_dir = '../data/verysmall/val'
#nb_train_samples = 144 #157
#nb_validation_samples = 16 #18
#total_num_classes = 2

epochs = 2
batch_size = 16

In [30]:

# build the VGG16 network
base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(150,150,3))
print('Model loaded.')

# build a classifier model to put on top of the convolutional model
"""
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
#top_model.add(Dropout(0.5))
top_model.add(Dense(total_num_classes, activation='sigmoid'))
"""


top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dropout(0.5))
top_model.add(Dense(1024, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(total_num_classes, activation='softmax'))



# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning
top_model.load_weights(top_model_weights_path)

# add the model on top of the convolutional base
# model.add(top_model)

model = Model(input= base_model.input, output= top_model(base_model.output))





def run_with_parameters(frozen_layers = 25, lr = 1e-4, momentum = 0.9, shear_range=0.2, zoom_range=0.2, horizontal_flip=True):
    # set the first 25 layers (up to the last conv block)
    # to non-trainable (weights will not be updated)
    for layer in model.layers:
        layer.trainable = True;
    
    for layer in model.layers[:frozen_layers]:
        layer.trainable = False

    # compile the model with a SGD/momentum optimizer
    # and a very slow learning rate.
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.SGD(lr=lr, momentum=momentum),
                  metrics=['accuracy'])

    # prepare data augmentation configuration
    train_datagen = ImageDataGenerator(
        rescale=1. / 255,
        shear_range=shear_range,
        zoom_range=zoom_range,
        horizontal_flip=horizontal_flip)

    test_datagen = ImageDataGenerator(rescale=1. / 255)

    train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical')

    validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical')

    # fine-tune the model
    model.fit_generator(
        train_generator,
        samples_per_epoch=nb_train_samples,
        epochs=epochs,
        validation_data=validation_generator,
        nb_val_samples=nb_validation_samples)


Model loaded.


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:33: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("se..., inputs=Tensor("in...)`


In [31]:
def run_with_parameters(frozen_layers = 25, lr = 1e-4, momentum = 0.9, shear_range=0.2, zoom_range=0.2, horizontal_flip=True):
    # set the first 25 layers (up to the last conv block)
    # to non-trainable (weights will not be updated)
    for layer in model.layers:
        layer.trainable = True;
    
    for layer in model.layers[:frozen_layers]:
        layer.trainable = False

    # compile the model with a SGD/momentum optimizer
    # and a very slow learning rate.
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.SGD(lr=lr, momentum=momentum),
                  metrics=['accuracy'])

    # prepare data augmentation configuration
    train_datagen = ImageDataGenerator(
        rescale=1. / 255,
        shear_range=shear_range,
        zoom_range=zoom_range,
        horizontal_flip=horizontal_flip)

    test_datagen = ImageDataGenerator(rescale=1. / 255)

    train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical')

    validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical')

    
    

    """
    weight_dir = "../model/%s"%pretrained_model
    if not os.path.exists(weight_dir):
        os.makedirs(weight_dir)
    weight_path = os.path.join(weight_dir, "best_weights%s.h5"%pretrained_model)
    checkpointer = keras.callbacks.ModelCheckpoint(filepath=weight_path), verbose=1,monitor='val_acc', save_best_only=True)
    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels),callbacks=[checkpointer])
    """
    
    pretrained_model = "temp_name"
    weight_dir = "../model/%s"%pretrained_model
    if not os.path.exists(weight_dir):
        os.makedirs(weight_dir)
    weight_path = os.path.join(weight_dir, "best_weights%s.h5"%pretrained_model)
    checkpointer = keras.callbacks.ModelCheckpoint(filepath=weight_path, verbose=1,monitor='val_acc', save_best_only=True, save_weights_only=True)
    # fine-tune the model
    model.fit_generator(
        train_generator,
        samples_per_epoch=nb_train_samples,
        epochs=epochs,
        validation_data=validation_generator,
        nb_val_samples=nb_validation_samples,
        callbacks=[checkpointer])


In [32]:

frozen_layers_arr = [25]
lr_arr = [1e-4]
momentum_arr = [0.9]
shear_range_arr = [0.2]
zoom_range_arr = [0.2]
horizontal_flip_arr = [True]

for frozen_layers_val in frozen_layers_arr:
    for lr_val in lr_arr:
        print("\n")
        print("frozen_layers_val = " + str(frozen_layers_val))
        print("lr_val = " + str(lr_val))
        run_with_parameters(frozen_layers = frozen_layers_val, lr = lr_val)

#run_with_parameters(frozen_layers = 25, lr = 1e-4, momentum = 0.9, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
    



frozen_layers_val = 25
lr_val = 0.0001
Found 1462 images belonging to 18 classes.
Found 167 images belonging to 18 classes.
Epoch 1/2
91/91 [==============================] - 38s - loss: 2.6238 - acc: 0.1662 - val_loss: 2.5787 - val_acc: 0.1569
Epoch 2/2
91/91 [==============================] - 38s - loss: 2.6420 - acc: 0.1667 - val_loss: 2.5814 - val_acc: 0.1565
Found 1462 images belonging to 18 classes.
Found 167 images belonging to 18 classes.
Epoch 1/2
91/91 [==============================] - 39s - loss: 2.6387 - acc: 0.1786 - val_loss: 2.5792 - val_acc: 0.1561
Epoch 2/2
91/91 [==============================] - 38s - loss: 2.6385 - acc: 0.1575 - val_loss: 2.5796 - val_acc: 0.1549


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:65: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., validation_data=<keras.pre..., steps_per_epoch=91, epochs=2, callbacks=[<keras.ca..., validation_steps=167)`
